In [5]:
import numpy as np 
import pandas as pd 
import os 
from sklearn.neighbors import NearestNeighbors
from pykdtree.kdtree import KDTree

In [6]:
def compute_normals_and_curvature_within_threshold(data, threshold):
    data = np.asarray(data) 
    nbrs = NearestNeighbors(radius=threshold).fit(data) 

    normals = np.zeros(data.shape) 
    curvature = np.zeros(data.shape[0]) 
    mean_curvature = np.zeros(data.shape[0]) 

    # Iterate over each point in the dataset
    for i, point in enumerate(data):
        # Find neighbors within the threshold
        distances, indices = nbrs.radius_neighbors([point], return_distance=True)

        # Flatten the list of indices and remove the point itself
        neighbors = data[indices[0]] 

        total_curvature = 0.0 

        # Skip points with insufficient neighbors (e.g., less than the dimensionality of space)
        if len(neighbors) < data.shape[1]:
            continue

        # Center the neighbors around the point
        centered_neighbors = neighbors - point

        # Perform PCA using Singular Value Decomposition (SVD)
        covariance_matrix = np.cov(centered_neighbors, rowvar=False)
        eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)  # Eigenvalues and eigenvectors

        # The normal vector is the eigenvector corresponding to the smallest eigenvalue
        normal_vector = eigenvectors[:, 0]
        normals[i] = normal_vector        

        # Compute curvature based on change in normal vector angles
        for neighbor in neighbors:
            if np.array_equal(point, neighbor):
                continue

            # Find neighbors of the current neighbor
            neighbor_distances, neighbor_indices = nbrs.radius_neighbors([neighbor], return_distance=True)
            secondary_neighbors = data[neighbor_indices[0]] - neighbor

            # Skip if there are insufficient neighbors for the neighbor point
            if len(secondary_neighbors) < data.shape[1]:
                continue

            # Fit PCA to secondary neighbors
            secondary_cov_matrix = np.cov(secondary_neighbors, rowvar=False)
            _, secondary_eigenvectors = np.linalg.eigh(secondary_cov_matrix)
            secondary_normal_vector = secondary_eigenvectors[:, 0]

            # Compute the angle between the normal vectors
            dot_product = np.dot(normal_vector, secondary_normal_vector)
            dot_product = np.clip(dot_product, -1.0, 1.0)  # Clip to avoid numerical issues
            angle_change = np.arccos(dot_product)

            # Estimate curvature based on the angle change and distance between the points
            distance_change = np.linalg.norm(neighbor - point)
            if distance_change > 0:
                curvature_point = angle_change / distance_change
                total_curvature += curvature_point
        
        curvature[i] = total_curvature 
        mean_curvature[i] = total_curvature / len(neighbors) 

        # print progress every 10% 
        if i % (data.shape[0] // 100) == 0:
            print(f"Progress: {i / data.shape[0] * 100:.2f}%")

    return normals, curvature, mean_curvature   

def compute_normals_and_curvature_knn(data, k):
    data = np.asarray(data) 
    nbrs = NearestNeighbors(n_neighbors=k+1).fit(data)  # +1 to include the point itself

    normals = np.zeros(data.shape) 
    curvature = np.zeros(data.shape[0]) 
    mean_curvature = np.zeros(data.shape[0]) 

    # Iterate over each point in the dataset
    for i, point in enumerate(data):
        # Find k-nearest neighbors
        distances, indices = nbrs.kneighbors([point])

        # Get the neighbors, excluding the point itself
        neighbors = data[indices[0][1:]]  # Exclude the first index, which is the point itself

        total_curvature = 0.0 

        # Skip points with insufficient neighbors (less than the dimensionality of space)
        if len(neighbors) < data.shape[1]:
            continue

        # Center the neighbors around the point
        centered_neighbors = neighbors - point

        # Perform PCA using Singular Value Decomposition (SVD)
        covariance_matrix = np.cov(centered_neighbors, rowvar=False)
        eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)  # Eigenvalues and eigenvectors

        # The normal vector is the eigenvector corresponding to the smallest eigenvalue
        normal_vector = eigenvectors[:, 0]
        normals[i] = normal_vector        

        # Compute curvature based on change in normal vector angles
        for neighbor in neighbors:
            if np.array_equal(point, neighbor):
                continue

            # Find k-nearest neighbors of the current neighbor
            neighbor_distances, neighbor_indices = nbrs.kneighbors([neighbor])
            secondary_neighbors = data[neighbor_indices[0][1:]] - neighbor  # Exclude the neighbor itself

            # Skip if there are insufficient neighbors for the neighbor point
            if len(secondary_neighbors) < data.shape[1]:
                continue

            # Fit PCA to secondary neighbors
            secondary_cov_matrix = np.cov(secondary_neighbors, rowvar=False)
            _, secondary_eigenvectors = np.linalg.eigh(secondary_cov_matrix)
            secondary_normal_vector = secondary_eigenvectors[:, 0]

            # Compute the angle between the normal vectors
            dot_product = np.dot(normal_vector, secondary_normal_vector)
            dot_product = np.clip(dot_product, -1.0, 1.0)  # Clip to avoid numerical issues
            angle_change = np.arccos(dot_product)

            # Estimate curvature based on the angle change and distance between the points
            distance_change = np.linalg.norm(neighbor - point)
            if distance_change > 0:
                curvature_point = angle_change / distance_change
                total_curvature += curvature_point
        
        curvature[i] = total_curvature 
        mean_curvature[i] = total_curvature / len(neighbors) 

        # Print progress every 10% 
        if i % (data.shape[0] // 100) == 0:
            print(f"Progress: {i / data.shape[0] * 100:.2f}%")

    return normals, curvature, mean_curvature


def compute_normals(data, k):
    data = np.asarray(data) 
    kdtree = KDTree(data) 
    print("Done building KDTree") 
    _, all_indices = kdtree.query(data, k=k+1) 
    print("Done querying KDTree") 
    normals = np.zeros(data.shape) 

    # Iterate over each point in the dataset
    for i, point in enumerate(data):
        # Find k-nearest neighbors
        # distances, indices = nbrs.kneighbors([point])
        indices = all_indices[i] 


        # Get the neighbors, excluding the point itself
        neighbors = data[indices]  # Exclude the first index, which is the point itself

        total_curvature = 0.0 

        # Skip points with insufficient neighbors (less than the dimensionality of space)
        if len(neighbors) < data.shape[1]:
            continue

        # Center the neighbors around the point
        centered_neighbors = neighbors - point

        # Perform PCA using Singular Value Decomposition (SVD)
        covariance_matrix = np.cov(centered_neighbors, rowvar=False)
        eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)  # Eigenvalues and eigenvectors

        # The normal vector is the eigenvector corresponding to the smallest eigenvalue
        normal_vector = eigenvectors[:, 0]
        normals[i] = normal_vector        

        # Print progress every 10% 
        if i % (data.shape[0] // 100) == 0:
            print(f"Progress: {i / data.shape[0] * 100:.2f}%")

    return normals 

In [7]:
# map = pd.read_csv('../data/extrusion_real_map.csv') 
# filepath = "/media/rp/Elements/abhay_ws/real_contact_data/slotted_circle_rounded/slotted_circle_aut_map_1.pkl"
filepath = "/media/rp/Elements/abhay_ws/mujoco_contact_graph_generation/results/cross_rounded_data/reverse_perturb_v4/processed_data/cross_rounded_peg_contact_map_sim.csv"
if filepath.endswith('.csv'):
    filetype = ".csv" 
    map = pd.read_csv(filepath) 
elif filepath.endswith('.pkl'): 
    filetype = ".pkl" 
    map = pd.read_pickle(filepath) 
map = map[['x', 'y', 'z', 'a', 'b', 'c']]

In [8]:
# threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5] 
# for threshold in threshold_list: 
#     normals, curvature, mean_curvature = compute_normals_and_curvature_within_threshold(map, threshold) 
#     map_normals_curvature = pd.DataFrame(data=np.concatenate((map, normals, curvature.reshape(-1, 1), mean_curvature.reshape(-1, 1)), axis=1), columns=['x', 'y', 'z', 'a', 'b', 'c', 'nx', 'ny', 'nz', 'na', 'nb', 'nc', 'curvature', 'mean_curvature']) 
#     fileout = filepath.replace(filetype, f"_with_normals_threshold_{threshold}.csv")    
#     map_normals_curvature.to_csv(fileout, index=False) 

k_neighbors_list = [10, 15, 20, 25] 
for k_neighbors in k_neighbors_list: 
    normals = compute_normals(map, k_neighbors) 
    map_normals_curvature = pd.DataFrame(data=np.concatenate((map, normals), axis=1), columns=['x', 'y', 'z', 'a', 'b', 'c', 'nx', 'ny', 'nz', 'na', 'nb', 'nc']) 
    fileout = filepath.replace(filetype, f"_with_normals_neighbors_{k_neighbors}.csv")   
    map_normals_curvature.to_csv(fileout, index=False)
    fileout = filepath.replace(filetype, f"_with_normals_neighbors_{k_neighbors}.pkl") 
    map_normals_curvature.to_pickle(fileout) 
    

Done building KDTree
Done querying KDTree
Progress: 0.00%
Progress: 1.00%
Progress: 2.00%
Progress: 3.00%
Progress: 4.00%
Progress: 5.00%
Progress: 6.00%
Progress: 7.00%
Progress: 8.00%
Progress: 9.00%
Progress: 10.00%
Progress: 11.00%
Progress: 12.00%
Progress: 13.00%
Progress: 14.00%
Progress: 15.00%
Progress: 16.00%
Progress: 17.00%
Progress: 18.00%
Progress: 19.00%
Progress: 20.00%
Progress: 21.00%
Progress: 22.00%
Progress: 23.00%
Progress: 24.00%
Progress: 25.00%
Progress: 26.00%
Progress: 27.00%
Progress: 28.00%
Progress: 29.00%
Progress: 30.00%
Progress: 31.00%
Progress: 32.00%
Progress: 33.00%
Progress: 34.00%
Progress: 35.00%
Progress: 36.00%
Progress: 37.00%
Progress: 38.00%
Progress: 39.00%
Progress: 40.00%
Progress: 41.00%
Progress: 42.00%
Progress: 43.00%
Progress: 44.00%
Progress: 45.00%
Progress: 46.00%
Progress: 47.00%
Progress: 48.00%
Progress: 49.00%
Progress: 50.00%
Progress: 51.00%
Progress: 52.00%
Progress: 53.00%
Progress: 54.00%
Progress: 55.00%
Progress: 56.00%

KeyboardInterrupt: 